In [1]:
import pandas as pd
import numpy as np
import os
import re

from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout

Using TensorFlow backend.


In [2]:
def load_data():
    
    dirs = os.listdir("c:/Users/patel/Documents/GitHub/wizardy-world-word-generation/books/")
    data = []
    
    for file_name in dirs:
        file_name = "./books/" + file_name
        file = open(file_name, 'r', encoding="utf8")
        data.append(file.read())
        file.close()
    
    return data

In [3]:
def clean_data(data):
    
    words = []
    for i in data:
        words.append(re.findall(r'\w+', i))
    cleaned_data = []
    for i in words:
        for word in i:
            if word.isalpha():
                cleaned_data.append(word.lower())
                
    return cleaned_data

In [4]:
def sequence_data(cleaned_data):
    
    x = len(cleaned_data)//51
    cleaned_data = cleaned_data[:51*x]
    sequences = [cleaned_data[i:i+51] for i in range(0,len(cleaned_data),51)]
    
    return sequences

In [5]:
def tokenize_data(sequenced_data):
    
    global tokenizer 
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sequenced_data)
    tokenized_data = tokenizer.texts_to_sequences(sequenced_data)
    
    return tokenized_data

In [6]:
def categorical_data(tokenized_data):

    size = len(tokenizer.word_index) + 1
    X,Y = [],[]
    for i in tokenized_data:
        X.append(i[:-1])
        Y.append(i[-1])
    X = np.array(X)
    Y = np.array(Y)
    Y = to_categorical(Y, num_classes = size)
    
    return X, Y

In [7]:
def create_model(X, Y):
    
    model = Sequential()
    model.add(Embedding(len(tokenizer.word_index) + 1, 50, input_length=X.shape[1]))
    model.add(LSTM(100, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(100))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))
    
    return model

In [8]:
data = load_data()
cleaned_data = clean_data(data)
sequenced_data = sequence_data(cleaned_data)
tokenized_data = tokenize_data(sequenced_data)
final_data = categorical_data(tokenized_data)
X, Y = final_data[0], final_data[1]
print(X.shape[0], X.shape[1], Y.shape[0])

23012 50 23012


In [9]:
model = create_model(X, Y)
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            1045350   
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 100)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 20907)            

In [10]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X, Y, batch_size = 256, epochs=200)

C:\ProgramData\Anaconda3\envs\tf-gpu\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/200
23012/23012 [==============================] - 9s 402us/step - loss: 7.6628 - accuracy: 0.0385
Epoch 2/200
23012/23012 [==============================] - 8s 342us/step - loss: 6.6732 - accuracy: 0.0454
Epoch 3/200
23012/23012 [==============================] - 8s 346us/step - loss: 6.6004 - accuracy: 0.0454
Epoch 4/200
23012/23012 [==============================] - 8s 351us/step - loss: 6.5388 - accuracy: 0.0454
Epoch 5/200
23012/23012 [==============================] - 8s 360us/step - loss: 6.4319 - accuracy: 0.0454
Epoch 6/200
23012/23012 [==============================] - 8s 369us/step - loss: 6.3695 - accuracy: 0.0452
Epoch 7/200
23012/23012 [==============================] - 8s 368us/step - loss: 6.2900 - accuracy: 0.0461
Epoch 8/200
23012/23012 [==============================] - 8s 365us/step - loss: 6.1909 - accuracy: 0.0491
Epoch 9/200
23012/23012 [==============================] - 8s 368us/step - loss: 6.1036 - accuracy: 0.0534
Epoch 10/200
23012/23012 [===========

23012/23012 [==============================] - 8s 364us/step - loss: 2.7649 - accuracy: 0.3444
Epoch 77/200
23012/23012 [==============================] - 8s 367us/step - loss: 2.7272 - accuracy: 0.3513
Epoch 78/200
23012/23012 [==============================] - 8s 365us/step - loss: 2.6996 - accuracy: 0.3546
Epoch 79/200
23012/23012 [==============================] - 8s 364us/step - loss: 2.6838 - accuracy: 0.3585
Epoch 80/200
23012/23012 [==============================] - 8s 363us/step - loss: 2.6537 - accuracy: 0.3672
Epoch 81/200
23012/23012 [==============================] - 10s 440us/step - loss: 2.6281 - accuracy: 0.3675
Epoch 82/200
23012/23012 [==============================] - 10s 447us/step - loss: 2.5952 - accuracy: 0.3761
Epoch 83/200
23012/23012 [==============================] - 9s 396us/step - loss: 2.5499 - accuracy: 0.3816
Epoch 84/200
23012/23012 [==============================] - 9s 383us/step - loss: 2.5257 - accuracy: 0.3843
Epoch 85/200
23012/23012 [=============

23012/23012 [==============================] - 9s 381us/step - loss: 1.2801 - accuracy: 0.6482
Epoch 152/200
23012/23012 [==============================] - 10s 444us/step - loss: 1.2668 - accuracy: 0.6534
Epoch 153/200
23012/23012 [==============================] - 9s 389us/step - loss: 1.2580 - accuracy: 0.6577
Epoch 154/200
23012/23012 [==============================] - 8s 367us/step - loss: 1.2434 - accuracy: 0.6599
Epoch 155/200
23012/23012 [==============================] - 8s 365us/step - loss: 1.2487 - accuracy: 0.6583
Epoch 156/200
23012/23012 [==============================] - 9s 386us/step - loss: 1.2769 - accuracy: 0.6507
Epoch 157/200
23012/23012 [==============================] - 8s 356us/step - loss: 1.2288 - accuracy: 0.6622
Epoch 158/200
23012/23012 [==============================] - 8s 359us/step - loss: 1.2106 - accuracy: 0.6677
Epoch 159/200
23012/23012 [==============================] - 8s 365us/step - loss: 1.1992 - accuracy: 0.6700
Epoch 160/200
23012/23012 [=====

In [ ]:
model.fit(X, Y, batch_size = 256, epochs=50)

Epoch 1/50
23012/23012 [==============================] - 8s 334us/step - loss: 0.8658 - accuracy: 0.7513
Epoch 2/50
23012/23012 [==============================] - 8s 338us/step - loss: 0.8503 - accuracy: 0.7557
Epoch 3/50
23012/23012 [==============================] - 8s 350us/step - loss: 0.8415 - accuracy: 0.7605
Epoch 4/50
23012/23012 [==============================] - 8s 359us/step - loss: 0.8388 - accuracy: 0.7584
Epoch 5/50
23012/23012 [==============================] - 8s 363us/step - loss: 0.8369 - accuracy: 0.7576
Epoch 6/50
23012/23012 [==============================] - 8s 367us/step - loss: 0.8262 - accuracy: 0.7627
Epoch 7/50
23012/23012 [==============================] - 8s 367us/step - loss: 0.8112 - accuracy: 0.7629
Epoch 8/50
23012/23012 [==============================] - 8s 364us/step - loss: 0.8122 - accuracy: 0.7635
Epoch 9/50
23012/23012 [==============================] - 8s 364us/step - loss: 0.8314 - accuracy: 0.7651
Epoch 10/50
23012/23012 [=====================

In [11]:
from pickle import dump
model.save('model.h5')
dump(tokenizer, open('tokenizer.pkl', 'wb'))